# 전략 패턴의 리팩토링

## 고전적인 전략
- ### 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만든다.<br> 전략을 이용하면 사용하는 크라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다.

#### 콘텍스트
- 일부 계산을 서로 다른 알고리즘을 구현하는 교환 가능한 컴포넌트에 위임함으로써 서비스를 제공한다.
#### 전략
- 여러 알고리즘을 구현하는 컴포넌트의 공통된 인터페이스.
#### 구체적인 전략
- 전략의 구상 서브클래스 중 하나.

In [6]:
#주문 할인에 대한 처리

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Cutomer', 'name fidelity')

class LineItem :

    def __init__(self, product, quantity, price) :
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self) :
        return self.price * self.quantity
    
class Order : # 콘텍스트
    def __init__(self, customer, cart, promotion = None) :
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self) :
        if not hasattr(self, '__total') :
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self) :
        if self.promotion is None :
            discount = 0
        else :
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self) :
        return f"<Order total : {self.total():.2f} due : {self.due():.2f}"
    

class Promotion(ABC) : # 전략 : 추상 베이스 클래스
    
    @abstractmethod
    def discount(self, order) :
        """할인액을 구체적인 숫자로 반환한다."""
        
class FidelityPromo(Promotion) : # 첫번째 구체적인 전략
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    def discount(self, order) : 
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion) : # 두번째 구체적인 전략
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    
    def discount(self, order) :
        discount = 0
        for item in order.cart :
            if item.quantity >= 20 :
                discount += item.total() * 0.1
        return discount
    
class LargeOrderPromo(Promotion) : # 세번째 구체적인 전략
    '''10종류 이상의 상품을 구입하면 전체 7% 할인 적용'''
    
    def discount(self, order) :
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10 :
            return order.total() * .07
        return 0



In [7]:
joe = Customer("John Doe", 0)
ann = Customer("Ann Smith", 1100)
cart = [LineItem("banana", 4, .5), LineItem("apple", 10, 1.5), LineItem("watermellon", 5, 5.0)]
print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))
banana_cart = [LineItem("banana", 30, .5), LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, BulkItemPromo()))
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, LargeOrderPromo()))
print(Order(joe, cart, LargeOrderPromo()))

<Order total : 42.00 due : 42.00
<Order total : 42.00 due : 39.90
<Order total : 30.00 due : 28.50
<Order total : 10.00 due : 9.30
<Order total : 42.00 due : 42.00


## 함수 지향 전략

In [8]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem :
    
    def __init__(self, product, quantity, price) :
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self) :
        return self.price * self.quantity
    
class Order : # 콘텍스트
    
    def __init__(self, customer, cart, promotion = None) :
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self) :
        if not hasattr(self, '__total') :
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self) :
        if self.promotion is None :
            discount = 0
        else :
            # class(자신)가 인수로 받은 함수가 class(자신)를 다시 인수로 받는 신기한 형태 
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self) :
        return f"<Order total : {self.total():.2f} due : {self.due():.2f}"
    
def fidelity_promo(order) :
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order) :
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart :
        if item.quantity >= 20 :
            discount += item.total() * 0.1
    return discount

def large_order_promo(order) :
    '''10종류 이상의 상품을 구입하면 전체 7% 할인 적용'''
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10 :
        return order.total() * .07
    return 0


### 똑같은 코드이고 똑같이 실행된다.
#### 하지만 함수 지향 전략이 가독성, 코드 길이 면에서 매우매우매우매우 좋다.
#### 또한 객체를 자주 생성하지 않아 메모리, 시간을 효율적으로 사용할 수 있다.

### 주의사항
#### order 가 실행될 때마다 구체 전략 클래스 객체를 생성하는 고전적인 방법과 달리 함수 전략에서는 함수 객체가 공유되고 있다는 것을 명심하자

In [9]:
joe = Customer("John Doe", 0)
ann = Customer("Ann Smith", 1100)
cart = [LineItem("banana", 4, .5), LineItem("apple", 10, 1.5), LineItem("watermellon", 5, 5.0)]
print(Order(joe, cart, fidelity_promo))
print(Order(ann, cart, fidelity_promo))
banana_cart = [LineItem("banana", 30, .5), LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, bulk_item_promo))
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, large_order_promo))
print(Order(joe, cart, large_order_promo))

<Order total : 42.00 due : 42.00
<Order total : 42.00 due : 39.90
<Order total : 30.00 due : 28.50
<Order total : 10.00 due : 9.30
<Order total : 42.00 due : 42.00


In [14]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order) :
    """ 최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(joe, cart, best_promo))

<Order total : 10.00 due : 9.30
<Order total : 30.00 due : 28.50
<Order total : 42.00 due : 42.00


## 모듈에서 전략 찾기

### 새로운 할인 함수를 추가할 경우 promos 를 갱신해줘야 하기 때문에 이는 오류가 발생할 여지가 있다.

### 해결책 :
### globals() 는 사용할 수 있는 전역 객체(함수도 포함)를 저장하고 있다.
### 함수 네이밍에 패턴을 줘서 해당 패턴을 갖는 함수를 가져오자

In [28]:
# 이 방식으로 promos 를 만들면 새로운 할인 함수가 추가될 때 promos 를 갱신해주지 않아도 자동 갱신된다.
# best_promo는 포함하지 않아야 무한 재귀를 피할 수 있다.
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

def best_promo(order) :
    """ 최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(joe, cart, best_promo))

<Order total : 10.00 due : 9.30
<Order total : 30.00 due : 28.50
<Order total : 42.00 due : 42.00


# singledispath 를 활용한 범용함수

### 오버로딩을 지원하지 않는 파이썬에서 범용함수를 만드는 법

In [20]:
from functools import singledispatch
from collections import abc
import numbers
import html

# singledispatch 데커레이터를 사용하면 범용함수가 된다.
@singledispatch
def htmlize(obj) :
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace("\n", "<br>\n")
    return f"<p>{content}</p>"

# int 보다 확장적인 number.Intergral을 사용하는 것을 권장한다. 이는 int의 가상 슈퍼클래스이다.
@htmlize.register(numbers.Integral)
def _(n) :
    return f"<pre>{n} (0x{n:x})</pre>"

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq) :
    inner = "</li>\n<li>".join(htmlize(item) for item in seq)
    return "<ul>\n<li>" + inner + "</li>\n</ul>"

In [21]:
htmlize({1,2,3})

'<pre>{1, 2, 3}</pre>'

In [22]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [23]:
htmlize('Heimlich & Co .\n- a game')

'<p>Heimlich &amp; Co .<br>\n- a game</p>'

In [24]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [25]:
print(htmlize(['alpha', 66, {3,2,1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


# 매개변수화된 데커레이터

In [22]:
registry = []

def register(func) :
    print(f"running register {func}")
    registry.append(func)
    return func

@register
def f1():
    print("running f1()")
    
print("running main()")
print("registry->", registry)

running register <function f1 at 0x10789d3a0>
running main()
registry-> [<function f1 at 0x10789d3a0>]


In [27]:
registry = set()

# 실제 decorator 를 매개변수로 가진 함수로 감쌈으로써 튜닝할 수 있다.
def register(active=True):
    
    # 실제 decorator
    def decorate(func):
        print(f"running register(active={active}) -> decorate({func})")
        if active :
            registry.add(func)
        else :
            registry.discard(func)
            
        return func
    return decorate

@register(active=False)
def f1():
    print("running f1()")
    
@register()
def f2():
    print("running f2()")
    
def f2():
    print("running f3()")
            
            

running register(active=False) -> decorate(<function f1 at 0x10789d430>)
running register(active=True) -> decorate(<function f2 at 0x10789d280>)
